In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
df.head()

,lang,art_name,entity,start,end,class1,classes2,text
0,BG,BG_670.txt,Запад,152,156,Antagonist,"[Conspirator, Instigator, Foreign Adversary]",Опитът на колективния Запад да „обезкърви Руси...
1,BG,BG_670.txt,САЩ,530,532,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
2,BG,BG_670.txt,НАТО,535,538,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
3,BG,BG_670.txt,Украйна,578,584,Antagonist,[Foreign Adversary],Опитът на колективния Запад да „обезкърви Руси...
4,BG,BG_670.txt,украински войници,633,649,Innocent,[Victim],Опитът на колективния Запад да „обезкърви Руси...


In [3]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)
df.loc[448]

lang                                                       EN
art_name                                     EN_UA_103861.txt
entity                                                Chinese
start                                                     791
end                                                       797
class1                                             Antagonist
classes2                                                [Spy]
text        The World Needs Peacemaker Trump Again \n\n by...
label                                                       0
input       The World Needs Peacemaker Trump Again  by Jef...
Name: 448, dtype: object

In [4]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)
    
    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]
    
    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[0])

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label                                                            0
input            Опитът на колективния Запад да „обезкърви Руси...
new_start_end                                           (151, 156)
Name: 0, dtype: object


In [5]:
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    #print(start,end)
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)
print(df['span_input'].loc[0])

Опитът на колективния Запад да „обезкърви Русия“ с ръцете на властите в Киев „се провали с гръм и трясък“ и скоро от Украйна ... Опитът на колективния [SPAN_START] Запад [SPAN_END] да „обезкърви Русия“ с ръцете на властите в Киев „се провали с гръм и трясък“ и скоро от Украйна няма да остане почти нищо, ако не започне процесът на разрешаване на този въоръжен конфликт. Тази гледна точка изрази пред ТАСС бившият началник на кабинета на държавния секретар на САЩ Колин Пауъл, пенсионирания полковник от армията на САЩ Лорънс Уилкерсън. "Подкрепата на САЩ, НАТО и други западни съюзници за войната в Украйна срещу Русия е безумна. Това води до смъртта на украински войници в името на загубена кауза, ако не вземете предвид печеленето на пари от американски и европейски военни изпълнители, както и бруталния опит да се обезкърви Русия чрез трети страни“, каза Уилкерсън. "Усилията за постигане на тази последна цел, при цялата ѝ подлост, се провалиха с гръм и трясък", заяви видният американски воене

In [6]:
print(len("[SPAN_START] "))
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

13


In [7]:
start,end = df['new_start_end'].loc[448]
print(start,end)
print(df['span_input'].loc[448][start:end])

798 805
Chinese


In [8]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

Using device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

print(num_added_toks)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


2


In [10]:
test_sent = 'This sentence has an important [SPAN_START] subject [SPAN_END] a a a'
tokenize = preprocess_function({'span_input' : test_sent})
print(tokenize)
print(tokenizer.batch_decode(tokenize['input_ids']))

{'input_ids': [0, 3293, 149357, 1556, 142, 5526, 250002, 28368, 250003, 10, 10, 10, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 4), (5, 13), (14, 17), (18, 20), (21, 30), (31, 43), (44, 51), (52, 62), (63, 64), (65, 66), (67, 68), (0, 0)]}
['<s>', 'This', 'sentence', 'has', 'an', 'important', '[SPAN_START]', 'subject', '[SPAN_END]', 'a', 'a', 'a', '</s>']


In [11]:
data = df.loc[ : , ['span_input','label','new_start_end','entity']]
data['tokenized']=data.apply(preprocess_function,axis=1)

In [12]:
#print(data.loc[0]['tokenized'])
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [13]:
print(data['entity'].loc[500])
inds = data['indexes'].loc[500]
toks = data['tokenized'].loc[500]
tokid = toks['input_ids']
a = list()
for i in inds:
    a.append(tokid[i])
print(tokenizer.batch_decode(a))

Boris Johnson
['Boris', 'Johnson']


In [14]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))
#print(tids[0],tatt[0])
#print(indexes[448])

2902 2902 2902


In [15]:
a = list()
for i in indexes[500]:
    a.append(tids[500][i])
print(a)
print(tokenizer.batch_decode(a))

[tensor(67151), tensor(59520)]
['Boris', 'Johnson']


In [16]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1  


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]: 
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i
        
    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])
print(len(sliced_ids[500]),len(tids[500]),Min,Max,len(tids[ind2]),ind2,key_inds[500])

511 588 67 512 67 956 [1, 2]


In [17]:
a = list()
for i in key_inds[500]:
    a.append(tids[500][i])
print(a)
print(tokenizer.batch_decode(a),df['entity'].loc[500])

[tensor(67151), tensor(59520)]
['Boris', 'Johnson'] Boris Johnson


In [18]:
print(sliced_ids[500],sliced_ids[500].flip(0))

tensor([     0,  67151,  59520,  35968,      7,  19507,    204,  95528,      7,
          4420,   1295,     70,  41210,     92,     26,    136,  17378, 102374,
        116000,    191,    111,  82739,   1221,    186,    204,  12986,  15663,
           538,  39746,    297,     26, 250002,  67151,  59520, 250003,     47,
         35968,  19507,    204,  95528,      7,   4420,   1295,     70,  41210,
            92,     26,     23,   7639,  34695,  22120,      7,      6,  55776,
         95954,    618,   8108,  12403,    214,     47,  82739, 127773,  14216,
         45956,  59520,  18925,  19676,     71,  36061,  19507,     44,  95528,
             7,   4420,   1295,     70,  41210,     92,     58,    111,   1631,
            47,  71864,     10,  50029,  59714,    402,    927,     23,  82739,
             5,   1913,   7639,  34695,  22120,      7,      6,  55776,  95954,
           618,     70,   7198,   1221,   1631,     19,     70,     57,   4134,
             9,   2175,     18,   2069, 

In [19]:
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)
#print(input_ids[300],attMask[300],inputIds.shape,attMask.shape)

In [20]:
a = list()
for i in key_inds[300]:
    a.append(inputIds[300][i])
print(a)
#print(tokenizer.batch_decode(a),df['entity'].loc[300])
#print(tokenizer.batch_decode(inputIds[300]))

[tensor(6), tensor(213542)]


In [21]:
inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y = data['label'].values
print(y,inputIds_np,attMask_np)

[0 0 0 ... 0 0 0] [[     0   1089  22617 ...      1      1      1]
 [     0   1089  22617 ...      1      1      1]
 [     0   1089  22617 ...      1      1      1]
 ...
 [     0   1343 212212 ...      5      2      1]
 [     0 189322     12 ...   1993      2      1]
 [     0   5832   9279 ...      1      1      1]] [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 0 0 0]]


In [22]:
from sklearn.model_selection import train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    inputIds_np, attMask_np, y, test_size=0.2, random_state=42
)

In [23]:
X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [24]:
# Create TensorDatasets
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_ids, X_train_mask, y_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y_test)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [25]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
optimizer = AdamW(model.parameters(), lr=8e-6)

In [26]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    # Initialize tqdm progress bar for training
    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
    
    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        #print(preds)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
        
        # Update tqdm description with current loss
        train_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    
    # Test phase
    model.eval()
    test_loss = 0 
    correct_test_predictions = 0
    total_test_predictions = 0
    
    # Initialize tqdm progress bar for test
    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
    
    with torch.no_grad():
        for batch in test_progress_bar:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            test_loss += loss.item()
            
            # Calculate test accuracy
            preds = torch.argmax(logits, dim=-1)
            correct_test_predictions += (preds == labels).sum().item()
            total_test_predictions += labels.size(0)
            
            # Update tqdm description with current loss
            test_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions
    
    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Training Epoch 1/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [02:09<00:00,  1.13it/s, loss=0.881]


Epoch 1/10
Training loss: 1.0459, Training accuracy: 0.4632


Test Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:09<00:00,  3.82it/s, loss=0.73]


Test loss: 0.9752, Test accuracy: 0.5422


Training Epoch 2/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [02:10<00:00,  1.12it/s, loss=0.88]


...

Test loss: 0.5928, Test accuracy: 0.8382


Training Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [02:09<00:00,  1.13it/s, loss=0.0336]


Epoch 10/10
Training loss: 0.1239, Training accuracy: 0.9660


Test Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:09<00:00,  3.81it/s, loss=0.84]

Test loss: 0.6217, Test accuracy: 0.8451
